ЛАБОРАТОРНА РОБОТА №2

ФБ-35 Кохта Андрій

Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow
від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі
та природу даних, над якими виконується аналітичні операції

In [1]:
import urllib.request
import pandas as pd
from datetime import datetime
import os
import re

Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [2]:
directory = './forlab2/'
all_files = {}
new_index = {1: 24, 2: 26, 3: 25, 4: 27, 5: 3, 6: 4, 7: 8, 8: 21, 9: 22,
10: 23, 11: 10, 12: 9, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16,
19: 17, 20: 18, 21: 19, 22: 20, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}

In [3]:
def clean_data(data):
    text = data.decode('utf-8', errors='ignore')
    text = re.sub(r'<tt><pre>', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    return text.encode('utf-8')

In [5]:
def file_download(url, index, new_i):
    vhi_url = urllib.request.urlopen(url)
    now = datetime.now()
    current_time = now.strftime("%d_%m_%Y_%H-%M-%S")
    file_name = f"vhi_id_{index}_time_{current_time}.csv"
    file_path = f"{directory}{file_name}"
    exists = [True for file in os.listdir(directory) if f"id_{new_i}_" in file]

    if not exists:
        out = open(file_path, 'wb')
        cleaned = clean_data(vhi_url.read())
        out.write(cleaned)
        out.close()
        new_name = f"vhi_id_{new_i}_time_{current_time}.csv"
        new_path = f"{directory}{new_name}"
        os.rename(file_path, new_path)
        all_files[new_i] = new_name
        print(f"VHI: {new_name} is downloaded.. ({index})")
    else:
        print(f"File with id:{new_i} already exists.. ({index})")

In [6]:
for key, value in new_index.items():
    file_download(f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={key}&year1=1981&year2=2024&type=Mean", key, value)
print("<---End for--->")

VHI: vhi_id_24_time_14_03_2025_12-50-36.csv is downloaded.. (1)
VHI: vhi_id_26_time_14_03_2025_12-50-38.csv is downloaded.. (2)
VHI: vhi_id_25_time_14_03_2025_12-50-39.csv is downloaded.. (3)
VHI: vhi_id_27_time_14_03_2025_12-50-41.csv is downloaded.. (4)
VHI: vhi_id_3_time_14_03_2025_12-50-42.csv is downloaded.. (5)
VHI: vhi_id_4_time_14_03_2025_12-50-43.csv is downloaded.. (6)
VHI: vhi_id_8_time_14_03_2025_12-50-45.csv is downloaded.. (7)
VHI: vhi_id_21_time_14_03_2025_12-50-46.csv is downloaded.. (8)
VHI: vhi_id_22_time_14_03_2025_12-50-48.csv is downloaded.. (9)
VHI: vhi_id_23_time_14_03_2025_12-50-49.csv is downloaded.. (10)
VHI: vhi_id_10_time_14_03_2025_12-50-51.csv is downloaded.. (11)
VHI: vhi_id_9_time_14_03_2025_12-50-52.csv is downloaded.. (12)
VHI: vhi_id_11_time_14_03_2025_12-50-53.csv is downloaded.. (13)
VHI: vhi_id_12_time_14_03_2025_12-50-55.csv is downloaded.. (14)
VHI: vhi_id_13_time_14_03_2025_12-50-56.csv is downloaded.. (15)
VHI: vhi_id_14_time_14_03_2025_12-50-5

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше
про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [8]:
def read_data():
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Region']
    all_dataframes = []

    for region_id, file_name in all_files.items():
        try:
            file_path = f"{directory}{file_name}"
            data_frame = pd.read_csv(file_path, names=headers)
            data_frame = data_frame[data_frame['VHI'] != -1]

            try:
                data_frame = data_frame[data_frame['Year'].astype(str).str.isdigit()]
                data_frame['Region'] = region_id
                all_dataframes.append(data_frame)
            except ValueError:
                print(f"Incorrect input in: {file_name} ")

        except Exception as error:
            print(f"Error during operation with file: {file_name}, {error}")
            continue

    data_line = pd.concat(all_dataframes, ignore_index=True)
    data_line['Year'] = data_line['Year'].astype(int)
    data_line['Region'] = data_line['Region'].astype(int)

    return data_line

In [9]:
vhi_data = read_data()

print(vhi_data.head(), '\n', vhi_data.tail())

   Year Week     SMN     SMT     VCI     TCI     VHI  Region
0  1982    1   0.053  260.31   45.01   39.46   42.23      24
1  1982    2   0.054  262.29   46.83   31.75   39.29      24
2  1982    3   0.055  263.82   48.13   27.24   37.68      24
3  1982    4   0.053  265.33   46.09   23.91   35.00      24
4  1982    5   0.050  265.66   41.46   26.65   34.06      24 
        Year Week     SMN     SMT     VCI     TCI     VHI  Region
60367  2024   48   0.128  270.55   64.97   25.53   45.25       5
60368  2024   49   0.115  269.06   60.12   27.24   43.68       5
60369  2024   50   0.104  267.75   55.24   25.89   40.57       5
60370  2024   51   0.094  266.45   51.16   24.29   37.72       5
60371  2024   52   0.093  266.38   54.22   21.11   37.66       5


Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові)

In [10]:
vhi_data['Region'] = vhi_data['Region'].replace({
    1: 'Vinnytsya', 2: 'Volyn', 3: "Dnipropetrovs'k", 4: "Donets'k", 5: 'Zhytomyr', 6: 'Transcarpathia', 7: 'Zaporizhzhya', 8: "Ivano-Frankivs'k", 9: 'Kiev City',
    10: 'Kiev', 11: 'Kirovohrad', 12: "Luhans'k", 13: "L'viv", 14: 'Mykolayiv', 15: 'Odessa', 16: 'Poltava', 17: 'Rivne', 18: "Sevastopol'",
    19: 'Sumy', 20: "Ternopil'", 21: 'Kharkiv', 22: 'Kherson', 23: "Khmel'nyts'kyy", 24: 'Cherkasy', 25: 'Chernivtsi', 26: 'Chernihiv', 27: 'Crimea',
})

In [11]:
# print(vhi_data['Region'].value_counts())
print(vhi_data.head(), '\n', vhi_data.tail())

   Year Week     SMN     SMT     VCI     TCI     VHI    Region
0  1982    1   0.053  260.31   45.01   39.46   42.23  Cherkasy
1  1982    2   0.054  262.29   46.83   31.75   39.29  Cherkasy
2  1982    3   0.055  263.82   48.13   27.24   37.68  Cherkasy
3  1982    4   0.053  265.33   46.09   23.91   35.00  Cherkasy
4  1982    5   0.050  265.66   41.46   26.65   34.06  Cherkasy 
        Year Week     SMN     SMT     VCI     TCI     VHI    Region
60367  2024   48   0.128  270.55   64.97   25.53   45.25  Zhytomyr
60368  2024   49   0.115  269.06   60.12   27.24   43.68  Zhytomyr
60369  2024   50   0.104  267.75   55.24   25.89   40.57  Zhytomyr
60370  2024   51   0.094  266.45   51.16   24.29   37.72  Zhytomyr
60371  2024   52   0.093  266.38   54.22   21.11   37.66  Zhytomyr


Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

In [12]:
def area_year(data, region, year):
    data_line = data[(data["Region"] == region) & (data["Year"] == year)]
    return data_line

print(area_year(vhi_data, 'Kiev', 2024))

       Year Week     SMN     SMT     VCI     TCI     VHI Region
24544  2024    1   0.060  258.78   36.98   47.72   42.35   Kiev
24545  2024    2   0.059  258.08   37.79   49.41   43.60   Kiev
24546  2024    3   0.064  258.42   43.74   48.44   46.08   Kiev
24547  2024    4   0.069  259.32   48.87   47.45   48.15   Kiev
24548  2024    5   0.076  261.74   52.27   43.73   47.99   Kiev
24549  2024    6   0.086  265.18   55.60   37.16   46.37   Kiev
24550  2024    7   0.105  269.40   61.75   26.61   44.15   Kiev
24551  2024    8   0.124  273.07   66.99   18.77   42.85   Kiev
24552  2024    9   0.146  276.26   72.42   13.59   43.00   Kiev
24553  2024   10   0.161  278.79   72.94   12.70   42.82   Kiev
24554  2024   11   0.175  280.71   71.83   13.15   42.49   Kiev
24555  2024   12   0.194  283.41   73.64   11.74   42.68   Kiev
24556  2024   13   0.210  285.52   74.58   11.49   43.04   Kiev
24557  2024   14   0.228  287.12   73.95   12.33   43.14   Kiev
24558  2024   15   0.253  289.27   74.70

o Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

In [163]:
def find_values(data, region, year1, year2):
    data_line = data[(data["Region"] == region) & (data["Year"].between(year1, year2))].copy()
    data_line['VHI'] = data_line['VHI'].astype(float)

    min_v = data_line['VHI'].min()
    max_v = data_line['VHI'].max()
    average = data_line['VHI'].mean()
    median = data_line['VHI'].median()

    return min_v, max_v, round(average, 2), round(median, 2)

In [164]:
minimum, maximum, average, medians = find_values(vhi_data, 'Poltava', 2017, 2018)
print(f"Мінімальне VHI: {minimum},\n Максимальне VHI: {maximum},\n Середнє VHI: {average},\n Медіана VHI: {medians}\n")
minimum, maximum, average, medians = find_values(vhi_data, 'Kiev', 2017, 2018)
print(f"Мінімальне VHI: {minimum},\n Максимальне VHI: {maximum},\n Середнє VHI: {average},\n Медіана VHI: {medians}")

Мінімальне VHI: 30.43,
 Максимальне VHI: 70.41,
 Середнє VHI: 45.68,
 Медіана VHI: 42.65

Мінімальне VHI: 31.95,
 Максимальне VHI: 65.38,
 Середнє VHI: 45.89,
 Медіана VHI: 43.94


o Ряд VHI за вказаний діапазон років для вказаних областей;

In [17]:
def find_values(data, regions, years):
    data_line = data[(data['Region'].isin(regions)) & (data['Year'].isin(years))]
    return data_line[['Year', 'Region', 'VHI']]

print(find_values(vhi_data, ['Kiev City', 'Chernihiv'], [2015, 2022]))

       Year     Region     VHI
3952   2015  Chernihiv   50.69
3953   2015  Chernihiv   50.77
3954   2015  Chernihiv   48.43
3955   2015  Chernihiv   47.89
3956   2015  Chernihiv   47.91
...     ...        ...     ...
26723  2022  Kiev City   40.12
26724  2022  Kiev City   42.05
26725  2022  Kiev City   41.30
26726  2022  Kiev City   39.95
26727  2022  Kiev City   38.27

[208 rows x 3 columns]


o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [28]:
def drought_by_percent(data, percent):
    data_line = data.copy()
    data_line['VHI'] = pd.to_numeric(data_line['VHI'], errors='coerce')
    all_regions = data_line['Region'].nunique()
    drought_years = []

    for year in range(data_line['Year'].min(), data_line['Year'].max() + 1):
        affected_data = data_line[(data_line['Year'] == year) & (data_line['VHI'] <= 15) & (data_line['VHI'] != -1)]
        affected_regions = affected_data['Region'].nunique()

        if affected_regions > all_regions * (percent / 100):
            drought_years.append(year)
            print(f"Drought year: {year}")
            print(affected_data[['Year', 'Region', 'VHI']])


drought_by_percent(vhi_data, 15)

Drought year: 2000
       Year       Region    VHI
979    2000     Cherkasy  14.64
980    2000     Cherkasy  11.82
981    2000     Cherkasy  10.81
982    2000     Cherkasy  10.68
983    2000     Cherkasy  12.30
984    2000     Cherkasy  14.24
16633  2000      Kharkiv  14.61
16634  2000      Kharkiv  11.33
16635  2000      Kharkiv   9.36
16636  2000      Kharkiv   9.45
16637  2000      Kharkiv   9.73
16638  2000      Kharkiv  11.45
16639  2000      Kharkiv  14.29
23339  2000         Kiev  12.51
23340  2000         Kiev  10.60
23341  2000         Kiev  11.20
23342  2000         Kiev  12.32
23343  2000         Kiev  14.65
25573  2000    Kiev City  14.89
25574  2000    Kiev City  12.76
25575  2000    Kiev City   7.81
25576  2000    Kiev City   6.49
25577  2000    Kiev City   6.58
25578  2000    Kiev City   6.71
25579  2000    Kiev City   7.56
25580  2000    Kiev City   9.25
25581  2000    Kiev City  10.94
25582  2000    Kiev City  12.28
43464  2000  Sevastopol'  13.14
43465  2000  Sevastop